In this excerises we are going to demonsrate how to extract raw sources of data from the internet. This will be split into two sections. First we will consider a simple case where data is already structured and stored in a file, we will show how to use Python code to download and manipulate this data directly. The second case is dealing with automated extraction of unstructured data from a web page. This process is called scraping.


Import the packages

In [3]:
import requests
from bs4 import BeautifulSoup

First lets look at the requests package and its functions. The most useful function of the requests package is 'get'. It performs a HTTP request to a specific webpage. 

**HTTP Request**

HTTP stands for HyperText Transfer Protocol and is used for transferring data over the internet. A HTTP request is essentially a way for a client (like your web browser or a Python script) to communicate with a server (where a website's data is stored). When you type a URL into your web browser, it is actually sending a HTTP request to a server.

These requests can be of different types, called _methods_, examples of which include GET, POST, PUT, DELETE, etc. These methods tell the server what kind of action the client wants to perform.

**GET Request**

The GET method is the most common HTTP request type. It is used to _retrieve_ information from a server. When you enter a URL into your browser, you're sending a GET request to the server asking it to send back some HTML content.


In Python, we can use the `requests` library to send a HTTP GET request.


As an introduction, say you are taking an organic chemistry course and there aren't enough exercises with solutions to practise on. You might find a webpage that contains lots of examples with individual files, it would be tedious to go through and download all of these manually so lets try and do it automatically. We will use the Sparr Group at Basel University as an example.

https://sparr.chemie.unibas.ch/en/teaching/


**Making a GET request:**

This is how you make a GET request to a website:

In [4]:
res = requests.get('https://sparr.chemie.unibas.ch/en/teaching/')

**Checking the status code:**

First we need to check if our request was successful. We can find this and other information about the status of our requestion using the 'status code'

The status code tells you about the status of your HTTP request. For example, a status code of 200 means that your request was successful, a status code of 404 means the resource was not found on the server.

A full list of HTTP response codes can be found here.

https://en.wikipedia.org/wiki/List_of_HTTP_status_codes


In [5]:
print(res.status_code)

200


The `get` method is used to send a GET request to a server. The URL of the desired resource is passed as an argument.

**Accessing the response:**

When we make a request to a web server, the server responds with data and some metadata like status code, content-type, etc. This data is stored in the response. 

* `.text` gives you the response from the server as a string:
* `.json()` gives you the response as a JSON object, if the response was in the format of JSON:

In [6]:
print(res.text)

<!DOCTYPE html>
<html lang="en-US" class="no-js">
<head>

<meta charset="utf-8">
<!-- 
	This website is powered by TYPO3 - inspiring people to share!
	TYPO3 is a free open source Content Management Framework initially created by Kasper Skaarhoj and licensed under GNU/GPL.
	TYPO3 is copyright 1998-2024 of Kasper Skaarhoj. Extensions are copyright of their respective owners.
	Information and contribution at https://typo3.org/
-->



<title>Teaching | Research Group Sparr | University of Basel</title>
<meta name="generator" content="TYPO3 CMS" />
<meta property="og:title" content="Teaching | Research Group Sparr | University of Basel" />
<meta property="og:type" content="article" />
<meta property="og:url" content="https://sparr.chemie.unibas.ch/en/teaching/" />
<meta name="twitter:card" content="summary" />


<link rel="stylesheet" href="/typo3conf/ext/easyweb/Resources/Public/HtmlTemplate/css/typo3.bundle.css?1707835207" media="all">
<link rel="stylesheet" href="/typo3conf/ext/curl_cont

**HTTP headers:**

HTTP headers allow the client and the server to pass additional information with the request or the response. Headers include information like content type of the response, date, status code, etc.


In [8]:
print(res.headers)

{'Date': 'Wed, 17 Apr 2024 15:08:17 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Content-Language': 'en', 'X-Frame-Options': 'SAMEORIGIN', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Referrer-Policy': 'strict-origin-when-cross-origin', 'Content-Encoding': 'gzip', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains'}


It is extremely important when writing requests into a piece of software that the code segment is included in a 'try and except' block. 

In the context of making HTTP requests using the `requests` library, there are several types of exceptions that can occur due to various reasons, such as:

1. `requests.exceptions.Timeout`: This exception is raised when a request times out.

2. `requests.exceptions.TooManyRedirects`: This error occurs if a request exceeds the configured number of maximum redirections.

3. `requests.exceptions.HTTPError`: This exception is raised for certain types of invalid HTTP responses, like a 404 not found or 500 internal error.

4. `requests.exceptions.RequestException`: This is a base exception from which all the above exceptions inherit, and it's raised for all other types of exceptions.

When making a request to a web server, it is always possible that the server might not respond, there could be a network problem, or we may not get the expected data back. In these situations, a Python script without error handling would simply crash and stop execution. To prevent this, we use `try` and `except` blocks when making HTTP requests. By doing this, we can catch these exceptions, handle them appropriately (possibly by just printing an error message), and continue with the rest of our code instead of having our entire application crash.

In [9]:
from requests.exceptions import RequestException

url = 'https://httpbin.org/get'

try:
    response = requests.get(url)
    response.raise_for_status()
except RequestException as err:
    print(f"An Error Occured: {err}")
else:
    print(response.text)

{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate, br", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.31.0", 
    "X-Amzn-Trace-Id": "Root=1-661fe5f6-7baa181a0e93947e330baec7"
  }, 
  "origin": "128.178.38.31", 
  "url": "https://httpbin.org/get"
}



In the above code, the `raise_for_status()` function is used to raise an `HTTPError` if an error occurs (i.e., if the HTTP request returned an unsuccessful status code). If the request is successful, it will print the response text. If an error occurs during the request, it will be handled, and the error message will be printed out without crashing the script.

Now we have access to the web page and its contents as a python object, we need to be able to find specific elements and do things with them. For example, we might want to find all images and download them. Web pages are structured in a language called HTML, it can be tricky to read and interpret so we use a package called beautifulsoup to convert this HTML in a machine handlable code called a parse tree.

First we need to create a BeautifulSoup object which we do as follows. The first argument is the raw HTML content, the second argument 'html.parser' is the html parser to parse the page.


In [10]:
soup = BeautifulSoup(res.text, 'html.parser')

You can navigate the BeautifulSoup parse tree using attributes like `.contents`, `.parent`, `.next_sibling`, `.prev_sibling`

Here is an example of using `.contents` which returns a list of all children of a tag:

In [11]:
first_link = soup.a
first_link_contents = first_link.contents
print(first_link_contents)

['How do I activate JavaScript in this web browser?']


You can search the BeautifulSoup parse tree using methods like `.find()`, `.find_all()`, `.find_next()`, `.find_previous()`

Here is an example of using `.find_all()` which returns all elements with a certain tag:

In [12]:
all_links = soup.find_all('a')
print(all_links)

[<a href="https://www.enable-javascript.com/en/" target="_blank">How do I activate JavaScript in this web browser?</a>, <a class="nav-link" href="/en/contact/">Contact</a>, <a aria-controls="desktop-search" aria-expanded="false" class="nav-link collapsed" data-toggle="collapse" href="#desktop-search" role="button">Search</a>, <a class="d-flex flex-column flex-grow-1 align-items-start" href="/en/">
<img alt="University of Basel" src="/typo3conf/ext/easyweb/Resources/Public/Images/Logo_Unibas_BraPan_EN.svg?1706717806"/>
</a>, <a href="/en/">Department<br/>
of Chemistry</a>, <a class="secondary-logo ml-auto maw-51" href="https://chemie.unibas.ch">
<img alt="Department
of Chemistry" class="logo-right" src="/typo3conf/ext/site_configurator/Resources/Public/Logos/Logo_Unibas_DepChe_EN.svg?1706717809"/>
</a>, <a href="/en/">Research Group Sparr</a>, <a aria-controls="active-sub-nav" aria-expanded="false" class="flex-fill toogle collapsed toggle-button maw-50" data-toggle="collapse" href="#act

**Accessing tag attributes:**

You can access the attributes of a HTML tag using indexing:


In [13]:
first_link = soup.a
href = first_link['href']
print(href)

https://www.enable-javascript.com/en/


TASK **Extracting all the URLs found within a page’s `<a>` tags:**
The attribute 'href' specifies a link to a webpage that is embedded into a segment of text. for example the HTML ```<a href="https://sparr.chemie.unibas.ch/en/teaching/">Visit Sparr Group</a>``` would display text 'Visit Sparr Group' which when clicked on would direct to the webpage. Here the 'a' represents a 'tag' which you can search for using .findall().

Your task is to use the `requests` library to send a GET request to 'https://example.com'. Parse the response text with BeautifulSoup and print out all the URLs found within a page's `<a>` tags.


In [ ]:
### Your Code Here ###

When we want to extract specific data from a webpage, it is essential to know the nested structure that is present in the original HTML as this structure will be reflected in the BeautifulSoup object. When inspecting a webpage HTML we might find that all the files we want to download are contained with in a <file>. Then we could simply use .findall() to locate all files of interest without complex navigation through the nested structure.

First, open up a webpage in Google Chrome. Let's use "https://google.com" for instance. 

Next, move your cursor to the element you are interested in. Right-click on that element, and in the dropdown menu, click on the "Inspect" option. This would open up the Developer Tools on the right-hand side or at the bottom of your browser depending on your settings.

The Developer Tools panel comprises of many different tools, but for our current purpose, we'll focus primarily on the "Elements" pane. This pane show us the HTML structure of the webpage.

In the Elements pane, you'll see a lot of HTML, which is the code used to create the structure and contents of the web page. The part of the page you right-clicked to inspect would be highlighted in the HTML on the panel. You can browse through the HTML to find other elements as well.

If you want to find a specific HTML element in the code, hover over the different parts of the HTML code in the "Elements" pane, corresponding sections of the web page will be highlighted. This can help you figure out which part of the HTML corresponds to the data you're interested in.

This process only covers the basic functionality needed for web scraping. Developer Tools in browsers are very robust and have many other functionalities that can be very helpful when building or debugging websites.

For inspecting web pages in other browsers like Firefox, Safari, or Internet Explorer, you can use similar steps but the exact navigation and layout of developer tools might vary slightly. If you're not sure, it's best to do a quick Google search for "How to inspect webpage in [Your Browser]". 

Quesstion 2: 

Use *requests* to access https://sparr.chemie.unibas.ch/en/teaching/ through a browser and inspect the page HTML. Under Group Seminar, there is a list of Excercises and Solutions. Your task is to find BeautifulSoup code which will access all of the exercises and solution You should end up with 465 links in total. Do this without referring to ChatGPT or a similar chatbot. You can use other resources such as the <a href="https://beautiful-soup-4.readthedocs.io/en/latest/">Beautiful Soup documentation</a> or Stack Exchange. You could do this by using hard coded naviations through the internal structure, but try to find a pattern that enables .findall() to directly retrieve the files. You do not need to download the files, just verify that you have found the links

In [ ]:
### Your Code Here ###

Question 3: 
Provide the HTML code to ChatGPT and ask it to find the links for you. Play around until you get the same answer as your manually implemented code.

In [ ]:
### ChatGPT's Code Here ###

Question 4

A large number of machine learning applications use SMILES as a data representation format. This format encodes a molecule as a string. Although excellent for machine learning the SMILES format is very difficult for humans to interpret and understand. To get around this, there exist several software tools which convert SMILES strings into an image of a molecular structure.

The most commonly used tool is CDK Depict. This is only avaliable as a Java package, so to access it through python code we must use the web page.

https://www.simolecule.com/cdkdepict/depict.html

The webpage takes a SMILES string as input and then executes some JavaScript code to generate an image. Navigate to this webpage and click on one of the pre-provided images. Doing this will take you to a new webpage which contains only the image you clicked on. Closely inspect the link to this new page. 

We will provide a function called *urllib.parse.urlencode()* which converts a python dictionary into a a query suitable for a URL format. Try it out below. Note how it converts '=' characters into '%3D'.

Your task is inspect the link for a molecule of your choice and construct a dictionary of parameters that can be converted to a string using the *urllib.parse.urlencode()*. You should then build a string which represents the link to the image for a given SMILES. Finally use what you have learned about the *requests* library to retrieve the image and display it. Use the code skeleton provided below.

In [15]:
import urllib.parse

params =    {
                'smiles': 'CN1C=NC2=C1C(=O)N(C(=O)N2C)C',
            }

print(urllib.parse.urlencode(params))


smiles=CN1C%3DNC2%3DC1C%28%3DO%29N%28C%28%3DO%29N2C%29C


In [17]:
import requests
from IPython.display import SVG, display

CDKDEPICTLINK = 'https://www.simolecule.com/cdkdepict/depict/bow'


def smiles_depict_url(smiles: str, format: str = 'svg') -> str:
    """
    Generate the URL for the depiction of a SMILES string.
    Args:
        smiles: smiles string to depict
        format: 'svg', 'pdf', 'png', etc.
    Returns:
        URL string
    """
   
    """
    Complete the params dictionary and encode it into a URL string with the urllib.parse.urlencode function
    Remember it must be a valid link to a cdkdepict URL, you can test your URL's by just navigating to the link in your browser
    Return the URL string
    """

    params = {
        'smi': smiles,
        # Your code here
        
    }
    params_str = urllib.parse.urlencode(params)

    # Your code here

def display_svg(url: str) -> None:
    # post a request to the link you construct. Remember to handle the cases where the response does not work as intented
    # Look at the response content to find the SVG data.

    # Use the display function to display the SVG data
    display(SVG(svg_data))


smiles = 'CCOC(=O)C1=CC=CC=C1C(=O)OCC' 
url = smiles_depict_url(smiles)
display_svg(url)

<IPython.core.display.SVG object>